In [1]:
import pandas as pd
from datetime import datetime, timedelta
from scipy.optimize import minimize
import numpy as np
import yfinance as yf
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import timedelta
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
# Function to get daily stock data for a given ticker
def get_stock_data(ticker, start_date, end_date):
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        print(f"Failed to download data for {ticker}: {e}")
        return None

# Read the ticker symbols from the CSV file
SP500_df = pd.read_csv("tickers.csv")
sp500_tickers = list(SP500_df["Symbol"].iloc[:504])

# Define the start and end date for the past ten years
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=10)

# Initialize a dictionary to store the stock data
SP500_stock_data = {}

# Keep trying until we get data for 30 tickers
while len(SP500_stock_data) < 30:
    # Randomly sample 1 ticker from the list
    random_ticker = np.random.choice(sp500_tickers, size=1, replace=False)[0]

    # If we already have this ticker, skip to the next iteration
    if random_ticker in SP500_stock_data:
        continue

    data = get_stock_data(random_ticker, start_date=start_date, end_date=end_date)

    # Validate that the data contains at least 10 years of data
    # Assuming at least 250 trading days in a year
    if data is not None and len(data) >= 250 * 10:
        SP500_stock_data[random_ticker] = data
    else:
        print(f"Insufficient data for {random_ticker}, trying another ticker.")

print("Stock data downloaded successfully for tickers:", list(SP500_stock_data.keys()))
stock_names = [key for key in SP500_stock_data]
SP500_result_dict = {}
for dataset_name, dataset in SP500_stock_data.items():
    # Assuming your data is sorted by date, you can calculate the logarithmic return
    dataset['rt'] = np.log(dataset['Close']) - np.log(dataset['Close'].shift(1))
    # Calculate the one-day lagged logarithmic return
    dataset['rt-1'] = dataset['rt'].shift(1)
    # Calculate the two-day lagged logarithmic return
    dataset['rt-2'] = np.log(dataset['Open'].shift(2)) - np.log(dataset['Close'].shift(1))
    # Calculate the three-day lagged logarithmic return
    dataset['rt-3'] = np.log(dataset['Open'].shift(3)) - np.log(dataset['Close'].shift(1))
    dataset['ct'] = np.log(dataset['Close']) - np.log(dataset['Open'])
    dataset['ct-1'] = np.log(dataset['Close']) - np.log(dataset['Open'].shift(1))
    dataset['ot'] = np.log(dataset['Open']) - np.log(dataset['Close'].shift(1))
    dataset['ht'] = np.log(dataset['High']) - np.log(dataset['Open'])
    dataset['ht-1'] = np.log(dataset['High'].shift(1)) - np.log(dataset['Open'].shift(1))
    dataset['ht-2'] = np.log(dataset['High'].shift(2)) - np.log(dataset['Open'].shift(2))
    dataset['ht-2'] = np.log(dataset['High'].shift(3)) - np.log(dataset['Open'].shift(3))
    dataset['lt'] = np.log(dataset['Low']) - np.log(dataset['Open'])
    dataset['lt-1'] = np.log(dataset['Low'].shift(1)) - np.log(dataset['Open'].shift(1))
    dataset['lt-2'] = np.log(dataset['Low'].shift(2)) - np.log(dataset['Open'].shift(2))
    dataset['lt-3'] = np.log(dataset['Low'].shift(3)) - np.log(dataset['Open'].shift(3))

    close_prices = dataset['Close']
    high_prices = dataset['High']
    low_prices = dataset['Low']
    prev_close = np.roll(close_prices, 1)
    true_range = np.maximum(
        np.abs(high_prices - low_prices),
        np.abs(prev_close - high_prices),
        np.abs(prev_close - low_prices)
    )
    dataset=dataset.reset_index()
    dataset["Date"]=pd.to_datetime(dataset["Date"])
    result = dataset.dropna()
    SP500_result_dict[dataset_name] = result

def create_rolling_windows(stock_df, num_windows=7, train_years=3, test_years=1):
    rolling_windows = []
    start_date = stock_df['Date'].min()

    for _ in range(num_windows):
        train_end_date = start_date + pd.DateOffset(years=train_years)
        test_end_date = train_end_date + pd.DateOffset(years=test_years)

        if test_end_date > stock_df['Date'].max():
            break  # Stop if we don't have enough data for the next window

        # Extract training and testing sets
        training_set = stock_df[(stock_df['Date'] >= start_date) & (stock_df['Date'] < train_end_date)]
        testing_set = stock_df[(stock_df['Date'] >= train_end_date) & (stock_df['Date'] < test_end_date)]

        # Store the pairs
        rolling_windows.append((training_set, testing_set))

        # Move the start date by one year for the next window
        start_date += pd.DateOffset(years=1)

    return rolling_windows

# Create a new dictionary with rolling window pairs for each stock
SP500_Train_Test_Dict = {stock_name: create_rolling_windows(stock_df) for stock_name, stock_df in SP500_result_dict.items()}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Insufficient data for FISV, trying another ticker.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['FBHS']: Exception('%ticker%: No timezone found, symbol may be delisted')


Insufficient data for FBHS, trying another ticker.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KSU']: Exception('%ticker%: No timezone found, symbol may be delisted')


Insufficient data for KSU, trying another ticker.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Stock data downloaded successfully for tickers: ['DPZ', 'AVGO', 'TSLA', 'NOW', 'DTE', 'ADBE', 'COO', 'DHI', 'ADSK', 'TRV', 'AES', 'RF', 'WMT', 'LNT', 'COF', 'EXR', 'PCAR', 'EIX', 'TMO', 'FE', 'RSG', 'DOV', 'AMT', 'VZ', 'NUE', 'OKE', 'WAB', 'BIIB', 'NTRS', 'MGM']


<h2>SP500</h2>

<b>random forest</b>

In [2]:
start_time = time.time()
target_column = 'rt'
rf_return={}
# Initialize a dictionary to store the best models and their performance for each stock
RF_performance = {}

for stock_name, windows in SP500_Train_Test_Dict.items():
    error_year=[]
    ret=[]
    # Initialize a list to store the best models and their metrics for the current stock
    stock_best_models = []
    param_grid = {
    'max_depth': [15],
    'min_samples_split': [15],
    'min_samples_leaf': [15],
    'n_estimators': [500]}
    # Loop over each pair of training/testing set
    years=[2017,2018,2019,2020,2021,2022]
    count=0
    for  i in windows:
        train_df, test_df= i
        # Prepare the training data
        X_train = train_df.iloc[:,7:].drop(columns=[target_column])
        y_train = train_df.iloc[:,7:][target_column]

        # Prepare the testing data
        X_test = test_df.iloc[:,7:].drop(columns=[target_column])
        y_test = test_df.iloc[:,7:][target_column]

        # Initialize the GridSearchCV object
        rf = RandomForestRegressor(random_state=0)
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

        # Fit the grid search to the data
        grid_search.fit(X_train, y_train)

        # Get the best estimator
        best_rf = grid_search.best_estimator_

        # Predict on the testing set with the best estimator
        y_pred = best_rf.predict(X_test)


        ret.append(y_pred)


        # Calculate the performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        # Assuming the target column is the price and we are interested in the direction of the change
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test.values))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': best_rf,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })
        error_year.append({years[count]:stock_best_models})
        print("the Prediction of year "+str(years[count])+" of " +stock_name+" is done")
        print(str(years[count])+":")
        print(stock_best_models)
        count+=1
    # Add the best models and their performance to the dictionary
    RF_performance[stock_name] = error_year
    rf_return[stock_name]=ret
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

the Prediction of year 2017 of DPZ is done
2017:
[{'model': RandomForestRegressor(max_depth=15, min_samples_leaf=15, min_samples_split=15,
                      n_estimators=500, random_state=0), 'mse': 6.497793467586848e-05, 'rmse': 0.00806088919883337, 'mae': 0.003231702808106048, 'hit_rate': 0.927710843373494}]
the Prediction of year 2018 of DPZ is done
2018:
[{'model': RandomForestRegressor(max_depth=15, min_samples_leaf=15, min_samples_split=15,
                      n_estimators=500, random_state=0), 'mse': 6.497793467586848e-05, 'rmse': 0.00806088919883337, 'mae': 0.003231702808106048, 'hit_rate': 0.927710843373494}, {'model': RandomForestRegressor(max_depth=15, min_samples_leaf=15, min_samples_split=15,
                      n_estimators=500, random_state=0), 'mse': 5.7213695468806956e-05, 'rmse': 0.007563973523803938, 'mae': 0.003533390494737521, 'hit_rate': 0.9362549800796812}]
the Prediction of year 2019 of DPZ is done
2019:
[{'model': RandomForestRegressor(max_depth=15, min

In [3]:
def calculate_averages(data):
    grouped_data = {}
    for stock in data:
        for year_data in data[stock]:
            for year, records in year_data.items():
                totals = {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0}
                count = len(records)
                for record in records:
                    for key in totals:
                        totals[key] += record[key]
                if year not in grouped_data:
                    grouped_data[year] = {key: 0 for key in totals}
                for key in totals:
                    grouped_data[year][key] += totals[key] / count
    for year in grouped_data:
        for key in grouped_data[year]:
            grouped_data[year][key] /= len(data)

    return grouped_data

<B>LSTM</B>

In [4]:
start_time = time.time()
target_column = 'rt'

# Initialize a dictionary to store the best models and their performance for each stock
LSTM_performance = {}
LSTM_stock_return = {}

# Define LSTM model architecture with new parameters
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape, dropout=0.4))
    model.add(LSTM(128, return_sequences=False, dropout=0.4))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer="Adam", loss='mean_squared_error')
    return model

# Loop over each stock
for stock_name, windows in SP500_Train_Test_Dict.items():
    stock_best_models = []
    ret = []
    years = [2017, 2018, 2019, 2020, 2021, 2022]
    for year, (train_df, test_df) in zip(years, windows):
        # Standardize the data
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df.iloc[:, 7:].drop(columns=[target_column]))
        y_train_scaled = scaler_y.fit_transform(train_df[[target_column]])
        X_test_scaled = scaler_X.transform(test_df.iloc[:, 7:].drop(columns=[target_column]))
        y_test_scaled = scaler_y.transform(test_df[[target_column]])

        # Reshape data for LSTM model
        X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

        # Define and fit the LSTM model
        lstm_model = create_lstm_model(X_train_scaled.shape[1:])
        lstm_model.fit(X_train_scaled, y_train_scaled, batch_size=32, epochs=64)

        # Predict on the test set
        y_pred_scaled = lstm_model.predict(X_test_scaled)

        # Inverse transform predictions and actual values to original scale
        y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()
        y_test = scaler_y.inverse_transform(y_test_scaled).flatten()

        ret.append(y_pred)

        # Calculate performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': lstm_model,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })

        print(f"Prediction for year {year} of {stock_name} is done.")
        print(f"{year}: {stock_best_models[-1]}")

    # Add the best models and their performance to the dictionary
    LSTM_performance[stock_name] = stock_best_models
    LSTM_stock_return[stock_name] = ret

end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Streaming output truncated to the last 5000 lines.
Epoch 56/64
24/24 [==============================] - 0s 5ms/step - loss: 0.2793
Epoch 57/64
24/24 [==============================] - 0s 6ms/step - loss: 0.3413
Epoch 58/64
24/24 [==============================] - 0s 5ms/step - loss: 0.2724
Epoch 59/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2822
Epoch 60/64
24/24 [==============================] - 0s 5ms/step - loss: 0.1940
Epoch 61/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2438
Epoch 62/64
24/24 [==============================] - 0s 5ms/step - loss: 0.2258
Epoch 63/64
24/24 [==============================] - 0s 6ms/step - loss: 0.2758
Epoch 64/64
8/8 [==============================] - 1s 3ms/step
Prediction for year 2020 of VZ is done.
2020: {'model': <keras.src.engine.sequential.Sequential object at 0x7e3d04b90fa0>, 'mse': 8.506894937704163e-06, 'rmse': 0.0029166581797845565, 'mae': 0.0021444797270086216, 'hit_rate': 0.9442231075697212}


<B>SVM</B>

In [5]:
start_time = time.time()
target_column = 'rt'

# Initialize a dictionary to store the best models and their performance for each stock
SVR_performance = {}
SVR_stock_return={}
# Loop over each stock
for stock_name, windows in SP500_Train_Test_Dict.items():
    stock_best_models = []
    ret=[]
    # Loop over each pair of training/testing set
    years = [2017, 2018, 2019, 2020, 2021, 2022]
    for year, (train_df, test_df) in zip(years, windows):
        # Standardize the data
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df.iloc[:, 7:].drop(columns=[target_column]))
        y_train_scaled = scaler_y.fit_transform(train_df[[target_column]])
        X_test_scaled = scaler_X.transform(test_df.iloc[:, 7:].drop(columns=[target_column]))
        y_test_scaled = scaler_y.transform(test_df[[target_column]])

        # Flatten y arrays
        y_train_scaled = y_train_scaled.ravel()
        y_test_scaled = y_test_scaled.ravel()

        # Define and fit the SVR model
        svr_model = SVR(C=1.0, epsilon=0.1)
        svr_model.fit(X_train_scaled, y_train_scaled)

        # Predict on the test set
        y_pred_scaled = svr_model.predict(X_test_scaled)

        # Inverse tray_prensform predictions and actual values to original scale
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        y_test = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

        ret.append(y_pred)


        # Calculate performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': svr_model,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })

        print(f"Prediction for year {year} of {stock_name} is done.")
        print(f"{year}: {stock_best_models[-1]}")

    # Add the best models and their performance to the dictionary
    SVR_performance[stock_name] = stock_best_models
    SVR_stock_return[stock_name] = ret
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Prediction for year 2017 of DPZ is done.
2017: {'model': SVR(), 'mse': 0.0001096930312838774, 'rmse': 0.010473444098474838, 'mae': 0.003758988619366655, 'hit_rate': 0.9558232931726908}
Prediction for year 2018 of DPZ is done.
2018: {'model': SVR(), 'mse': 8.924297345871556e-05, 'rmse': 0.009446849922525263, 'mae': 0.003734323750727445, 'hit_rate': 0.9721115537848606}
Prediction for year 2019 of DPZ is done.
2019: {'model': SVR(), 'mse': 0.0004860135961533107, 'rmse': 0.022045716049911162, 'mae': 0.006822349663366609, 'hit_rate': 0.9365079365079365}
Prediction for year 2020 of DPZ is done.
2020: {'model': SVR(), 'mse': 2.5208847661475262e-05, 'rmse': 0.0050208413300437274, 'mae': 0.0022146853720928185, 'hit_rate': 0.9800796812749004}
Prediction for year 2021 of DPZ is done.
2021: {'model': SVR(), 'mse': 9.92807510861887e-05, 'rmse': 0.009963972655833049, 'mae': 0.0028213809849720255, 'hit_rate': 0.9484126984126984}
Prediction for year 2022 of DPZ is done.
2022: {'model': SVR(), 'mse': 0

<B>BPNN</B>

In [6]:
start_time = time.time()
target_column = 'rt'

# Initialize a dictionary to store the best models and their performance for each stock
BPNN_performance = {}
BPNN_stock_return = {}

# Loop over each stock
for stock_name, windows in SP500_Train_Test_Dict.items():
    stock_best_models = []
    ret = []

    # Loop over each pair of training/testing set
    years = [2017, 2018, 2019, 2020, 2021, 2022]
    for year, (train_df, test_df) in zip(years, windows):
        # Standardize the data
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df.iloc[:, 7:].drop(columns=[target_column]))
        y_train_scaled = scaler_y.fit_transform(train_df[[target_column]])
        X_test_scaled = scaler_X.transform(test_df.iloc[:, 7:].drop(columns=[target_column]))
        y_test_scaled = scaler_y.transform(test_df[[target_column]])

        # Flatten y arrays
        y_train_scaled = y_train_scaled.ravel()
        y_test_scaled = y_test_scaled.ravel()

        # Define and fit the BPNN model
        bpnn_model = Sequential()
        bpnn_model.add(Dense(10, input_dim=X_train_scaled.shape[1], activation='relu'))  # Adjust number of neurons
        bpnn_model.add(Dense(1, activation='linear'))
        bpnn_model.compile(loss='mean_squared_error', optimizer='adam')
        bpnn_model.fit(X_train_scaled, y_train_scaled, epochs=64, batch_size=32, verbose=0)  # Adjust epochs and batch_size

        # Predict on the test set
        y_pred_scaled = bpnn_model.predict(X_test_scaled)

        # Inverse transform predictions and actual values to original scale
        y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()
        y_test = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

        ret.append(y_pred)

        # Calculate performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': bpnn_model,
            'mse': mse,

            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })

        print(f"Prediction for year {year} of {stock_name} is done.")
        print(f"{year}: {stock_best_models[-1]}")

    # Add the best models and their performance to the dictionary
    BPNN_performance[stock_name] = stock_best_models
    BPNN_stock_return[stock_name] = ret

end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

8/8 [==============================] - 0s 2ms/step
Prediction for year 2017 of DPZ is done.
2017: {'model': <keras.src.engine.sequential.Sequential object at 0x7e408976f040>, 'mse': 4.4395554142293885e-06, 'rmse': 0.002107025252394804, 'mae': 0.0016082620500313068, 'hit_rate': 0.9437751004016064}
8/8 [==============================] - 0s 2ms/step
Prediction for year 2018 of DPZ is done.
2018: {'model': <keras.src.engine.sequential.Sequential object at 0x7e4088930d00>, 'mse': 8.849936187826866e-06, 'rmse': 0.002974884230995698, 'mae': 0.002187514585218079, 'hit_rate': 0.9442231075697212}
8/8 [==============================] - 0s 2ms/step
Prediction for year 2019 of DPZ is done.
2019: {'model': <keras.src.engine.sequential.Sequential object at 0x7e3c8cd4bf40>, 'mse': 2.3576343815534195e-05, 'rmse': 0.004855547735892851, 'mae': 0.002719349542973096, 'hit_rate': 0.9563492063492064}
8/8 [==============================] - 0s 2ms/step
Prediction for year 2020 of DPZ is done.
2020: {'model': <

<B>ELM</B>

In [7]:
class ELMRegressor:
    def __init__(self, n_hidden_units):
        self.n_hidden_units = n_hidden_units

    def fit(self, X, y):
        self.input_weights = np.random.normal(size=[X.shape[1], self.n_hidden_units])
        self.biases = np.random.normal(size=[self.n_hidden_units])

        # Calculate hidden layer output
        H = np.tanh(np.dot(X, self.input_weights) + self.biases)

        # Calculate output weights (beta)
        self.beta = np.dot(np.linalg.pinv(H), y)

    def predict(self, X):
        H = np.tanh(np.dot(X, self.input_weights) + self.biases)
        return np.dot(H, self.beta)
start_time = time.time()
target_column = 'rt'

# Initialize a dictionary to store the best models and their performance for each stock
ELM_performance = {}
ELM_stock_return={}
# Loop over each stock
for stock_name, windows in SP500_Train_Test_Dict.items():
    stock_best_models = []
    ret = []

    # Loop over each pair of training/testing set
    years = [2017, 2018, 2019, 2020, 2021, 2022]
    for year, (train_df, test_df) in zip(years, windows):
        # Standardize the data
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df.iloc[:, 7:].drop(columns=[target_column]))
        y_train_scaled = scaler_y.fit_transform(train_df[[target_column]])
        X_test_scaled = scaler_X.transform(test_df.iloc[:, 7:].drop(columns=[target_column]))
        y_test_scaled = scaler_y.transform(test_df[[target_column]])

        # Flatten y arrays
        y_train_scaled = y_train_scaled.ravel()
        y_test_scaled = y_test_scaled.ravel()

        # Define and fit the ELM model
        elm_model = ELMRegressor(n_hidden_units=100)
        elm_model.fit(X_train_scaled, y_train_scaled)

        # Predict on the test set
        y_pred_scaled = elm_model.predict(X_test_scaled)

        # Inverse transform predictions and actual values to original scale
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        y_test = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

        ret.append(y_pred)
        # Calculate performance metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Hit Rate
        direction_pred = np.sign(np.diff(y_pred))
        direction_true = np.sign(np.diff(y_test))
        hit_rate = np.mean(direction_pred == direction_true)

        # Store the best model and its performance metrics
        stock_best_models.append({
            'model': elm_model,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'hit_rate': hit_rate
        })

        print(f"Prediction for year {year} of {stock_name} is done.")
        print(f"{year}: {stock_best_models[-1]}")

    # Add the best models and their performance to the dictionary
    ELM_performance[stock_name] = stock_best_models
    ELM_stock_return[stock_name] = ret
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Prediction for year 2017 of DPZ is done.
2017: {'model': <__main__.ELMRegressor object at 0x7e3c8c613190>, 'mse': 5.7437707462033474e-05, 'rmse': 0.007578766882681738, 'mae': 0.005060643339638788, 'hit_rate': 0.8955823293172691}
Prediction for year 2018 of DPZ is done.
2018: {'model': <__main__.ELMRegressor object at 0x7e3c143700a0>, 'mse': 6.057240383136127e-05, 'rmse': 0.007782827495927252, 'mae': 0.00566455529943028, 'hit_rate': 0.900398406374502}
Prediction for year 2019 of DPZ is done.
2019: {'model': <__main__.ELMRegressor object at 0x7e3c2058bee0>, 'mse': 0.0002917082799620373, 'rmse': 0.017079469545686638, 'mae': 0.007664486153867865, 'hit_rate': 0.9087301587301587}
Prediction for year 2020 of DPZ is done.
2020: {'model': <__main__.ELMRegressor object at 0x7e3c14371180>, 'mse': 3.633888695466245e-05, 'rmse': 0.006028174429681216, 'mae': 0.004580464987555042, 'hit_rate': 0.8844621513944223}
Prediction for year 2021 of DPZ is done.
2021: {'model': <__main__.ELMRegressor object at

In [8]:
def calculate_average_metrics(data):
    averages = {}
    for stock, records in data.items():
        total_metrics = {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0}
        for record in records:
            total_metrics['mse'] += record['mse']
            total_metrics['rmse'] += record['rmse']
            total_metrics['mae'] += record['mae']
            total_metrics['hit_rate'] += record['hit_rate']
        averages[stock] = {metric: total / len(records) for metric, total in total_metrics.items()}

    return averages

def calculate_average_by_year(data):
    year_metrics = {2017: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2018: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2019: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2020: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2021: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2022: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0}}
    counts = {year: 0 for year in year_metrics}
    for stock, records in data.items():
        for index, record in enumerate(records):
            year = 2017 + index
            # 累加指标
            year_metrics[year]['mse'] += record['mse']
            year_metrics[year]['rmse'] += record['rmse']
            year_metrics[year]['mae'] += record['mae']
            year_metrics[year]['hit_rate'] += record['hit_rate']
            counts[year] += 1
    for year in year_metrics:
        for metric in year_metrics[year]:
            year_metrics[year][metric] /= counts[year]

    return year_metrics
# calculate mean
RF_averages = calculate_averages(RF_performance)
rf=pd.DataFrame(RF_averages)
rf["model"]="rf"
LSTM_averages = calculate_average_by_year(LSTM_performance)
lstm=pd.DataFrame(LSTM_averages)
lstm["model"]="lstm"
SVR_averages = calculate_average_by_year(SVR_performance)
svr=pd.DataFrame(SVR_averages)
svr["model"]="svr"
BPNN_averages = calculate_average_by_year(BPNN_performance)
bpnn=pd.DataFrame(BPNN_averages)
bpnn["model"]="bpnn"
ELM_averages = calculate_average_by_year(ELM_performance)
elm=pd.DataFrame(ELM_averages)
elm["model"]="elm"

In [9]:
df_models = pd.concat([rf, lstm, svr, bpnn, elm])
for name, group in df_models.groupby(by=["model"])[2017]:
    print(name)
    print(group)

bpnn
mse         0.000006
rmse        0.002281
mae         0.001639
hit_rate    0.945515
Name: 2017, dtype: float64
elm
mse         0.000041
rmse        0.006104
mae         0.003849
hit_rate    0.908300
Name: 2017, dtype: float64
lstm
mse         0.000023
rmse        0.004578
mae         0.002788
hit_rate    0.934404
Name: 2017, dtype: float64
rf
mse         0.000131
rmse        0.009220
mae         0.003686
hit_rate    0.950679
Name: 2017, dtype: float64
svr
mse         0.000051
rmse        0.006270
mae         0.002072
hit_rate    0.960509
Name: 2017, dtype: float64


In [10]:
df_models = pd.concat([rf, lstm, svr, bpnn, elm])
for name, group in df_models.groupby(by=["model"])[2018]:
    print(name)
    print(group)

bpnn
mse         0.000010
rmse        0.002932
mae         0.002061
hit_rate    0.941036
Name: 2018, dtype: float64
elm
mse         0.000075
rmse        0.007661
mae         0.004724
hit_rate    0.908367
Name: 2018, dtype: float64
lstm
mse         0.000038
rmse        0.005569
mae         0.003415
hit_rate    0.928552
Name: 2018, dtype: float64
rf
mse         0.000131
rmse        0.009220
mae         0.003686
hit_rate    0.950679
Name: 2018, dtype: float64
svr
mse         0.000113
rmse        0.008771
mae         0.003207
hit_rate    0.955113
Name: 2018, dtype: float64


In [11]:
df_models = pd.concat([rf, lstm, svr, bpnn, elm])
for name, group in df_models.groupby(by=["model"])[2019]:
    print(name)
    print(group)

bpnn
mse         0.000026
rmse        0.004798
mae         0.002582
hit_rate    0.947884
Name: 2019, dtype: float64
elm
mse         0.000426
rmse        0.019231
mae         0.007797
hit_rate    0.913360
Name: 2019, dtype: float64
lstm
mse         0.000254
rmse        0.014480
mae         0.005958
hit_rate    0.931614
Name: 2019, dtype: float64
rf
mse         0.000131
rmse        0.009220
mae         0.003686
hit_rate    0.950679
Name: 2019, dtype: float64
svr
mse         0.000645
rmse        0.024056
mae         0.007481
hit_rate    0.937037
Name: 2019, dtype: float64


In [12]:
df_models = pd.concat([rf, lstm, svr, bpnn, elm])
for name, group in df_models.groupby(by=["model"])[2020]:
    print(name)
    print(group)

bpnn
mse         0.000046
rmse        0.005872
mae         0.004176
hit_rate    0.933466
Name: 2020, dtype: float64
elm
mse         0.000249
rmse        0.013896
mae         0.009660
hit_rate    0.860823
Name: 2020, dtype: float64
lstm
mse         0.000073
rmse        0.007648
mae         0.005077
hit_rate    0.938911
Name: 2020, dtype: float64
rf
mse         0.000131
rmse        0.009220
mae         0.003686
hit_rate    0.950679
Name: 2020, dtype: float64
svr
mse         0.000297
rmse        0.015016
mae         0.006659
hit_rate    0.940770
Name: 2020, dtype: float64


In [13]:
df_models = pd.concat([rf, lstm, svr, bpnn, elm])
for name, group in df_models.groupby(by=["model"])[2021]:
    print(name)
    print(group)

bpnn
mse         0.000014
rmse        0.003469
mae         0.002564
hit_rate    0.937037
Name: 2021, dtype: float64
elm
mse         0.000087
rmse        0.008720
mae         0.006300
hit_rate    0.885847
Name: 2021, dtype: float64
lstm
mse         0.000031
rmse        0.005200
mae         0.003634
hit_rate    0.936905
Name: 2021, dtype: float64
rf
mse         0.000131
rmse        0.009220
mae         0.003686
hit_rate    0.950679
Name: 2021, dtype: float64
svr
mse         0.000048
rmse        0.005185
mae         0.002091
hit_rate    0.966667
Name: 2021, dtype: float64


In [14]:
df_models = pd.concat([rf, lstm, svr, bpnn, elm])
for name, group in df_models.groupby(by=["model"])[2022]:
    print(name)
    print(group)

bpnn
mse         0.000020
rmse        0.004223
mae         0.003075
hit_rate    0.937483
Name: 2022, dtype: float64
elm
mse         0.000113
rmse        0.010215
mae         0.007444
hit_rate    0.887416
Name: 2022, dtype: float64
lstm
mse         0.000044
rmse        0.006397
mae         0.004567
hit_rate    0.935207
Name: 2022, dtype: float64
rf
mse         0.000131
rmse        0.009220
mae         0.003686
hit_rate    0.950679
Name: 2022, dtype: float64
svr
mse         0.000077
rmse        0.007592
mae         0.002982
hit_rate    0.962651
Name: 2022, dtype: float64


In [15]:
import numpy as np

# Number of simulations
num_simulations = 100

# Initialize a dictionary to store all predictions for each stock
all_predictions = {stock_name: [] for stock_name in SP500_Train_Test_Dict.keys()}

# Monte Carlo simulation: repeat the prediction process 5000 times
for _ in range(num_simulations):
    for stock_name, windows in SP500_Train_Test_Dict.items():
        # Select the most recent testing set as the future data to predict
        _, most_recent_test_df = windows[-1]  # Last pair in the list is the most recent

        # Standardize the feature columns of the most recent data
        X_future_scaled = scaler_X.transform(most_recent_test_df.iloc[:, 7:].drop(columns=[target_column]))

        # Use the last trained BPNN model for this stock to make predictions on the future data
        last_model = BPNN_performance[stock_name][-1]['model']  # Last model in the list for the stock
        future_pred_scaled = last_model.predict(X_future_scaled)

        # Inverse transform the predictions to the original scale
        future_pred = scaler_y.inverse_transform(future_pred_scaled).flatten()

        # Store the predictions
        all_predictions[stock_name].append(future_pred)

8/8 [==============================] - 0s 2ms/step


In [16]:
def calculate_average_metrics(data):
    averages = {}
    for stock, records in data.items():
        # 初始化累计指标
        total_metrics = {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0}

        # 累加每条记录的指标
        for record in records:
            total_metrics['mse'] += record['mse']
            total_metrics['rmse'] += record['rmse']
            total_metrics['mae'] += record['mae']
            total_metrics['hit_rate'] += record['hit_rate']

        # 计算平均值
        averages[stock] = {metric: total / len(records) for metric, total in total_metrics.items()}

    return averages

def calculate_average_by_year(data):
    year_metrics = {2017: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2018: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2019: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2020: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2021: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0},
                    2022: {'mse': 0, 'rmse': 0, 'mae': 0, 'hit_rate': 0}}
    counts = {year: 0 for year in year_metrics}
    for stock, records in data.items():
        for index, record in enumerate(records):
            year = 2017 + index
            # 累加指标
            year_metrics[year]['mse'] += record['mse']
            year_metrics[year]['rmse'] += record['rmse']
            year_metrics[year]['mae'] += record['mae']
            year_metrics[year]['hit_rate'] += record['hit_rate']
            counts[year] += 1
    for year in year_metrics:
        for metric in year_metrics[year]:
            year_metrics[year][metric] /= counts[year]

    return year_metrics
# calculate mean
RF_averages = calculate_averages(RF_performance)
rf=pd.DataFrame(RF_averages)
rf["model"]="rf"
LSTM_averages = calculate_average_by_year(LSTM_performance)
lstm=pd.DataFrame(LSTM_averages)
lstm["model"]="lstm"
SVR_averages = calculate_average_by_year(SVR_performance)
svr=pd.DataFrame(SVR_averages)
svr["model"]="svr"
BPNN_averages = calculate_average_by_year(BPNN_performance)
bpnn=pd.DataFrame(BPNN_averages)
bpnn["model"]="bpnn"
ELM_averages = calculate_average_by_year(ELM_performance)
elm=pd.DataFrame(ELM_averages)
elm["model"]="elm"

In [17]:
mean_future_returns = {}
for stock_name, preds in all_predictions.items():
  lst=[]
  for pred in preds:
    re=1
    for i in pred:
      re*=(1+i)
    lst.append(re)
  mean_future_returns[stock_name] = np.mean(lst)
# Calculate the mean predicted return for each stock across all simulations


# Sort the stocks by their mean predicted return in descending order
sorted_stocks_by_return = sorted(mean_future_returns.items(), key=lambda x: x[1], reverse=True)

# Select the top eight stocks
top_eight_stocks = sorted_stocks_by_return[:8]

# Extract just the stock names if needed
top_eight_stock_names = [stock[0] for stock in top_eight_stocks]

print("Top eight stocks based on mean predicted future returns across 1000 simulations:")
for stock, mean_return in top_eight_stocks:
    print(f"{stock}: {mean_return:.4f}")

Top eight stocks based on mean predicted future returns across 1000 simulations:
WAB: 1.5209
LNT: 1.2378
VZ: 1.2293
TRV: 1.2225
EIX: 1.1416
DHI: 1.1328
WMT: 1.1326
RSG: 1.0709


In [18]:
for stock_name, preds in all_predictions.items():
  lst=[]
  for pred in preds:
    re=1
    for i in pred:
      re*=(1+i)
    lst.append(re)
  mean_future_returns[stock_name] = np.mean(lst)

In [19]:
mean_returns=[]
stocks=[]
for stock, ret in top_eight_stocks:
  mean_returns.append(ret)
  stocks.append(stock)

In [20]:
future_return=pd.DataFrame()
for i in stocks:
  future_return[i]=np.mean(all_predictions[i], axis=0)
future_return

,WAB,LNT,VZ,TRV,EIX,DHI,WMT,RSG
0,-0.018220,-0.009445,0.008773,-0.012111,0.001418,-0.000759,-0.000170,-0.009315
1,-0.038437,0.014720,0.014686,-0.000396,-0.004216,-0.027202,0.005824,0.011997
2,0.011795,0.029273,0.012941,0.006537,0.012942,-0.039964,0.026164,0.001009
3,-0.000069,-0.005438,-0.007818,-0.003822,-0.010678,-0.011430,0.015157,0.015657
4,-0.009145,0.006454,0.022891,0.016665,0.014258,0.003473,0.009026,0.005056
...,...,...,...,...,...,...,...,...
245,0.021856,-0.006781,0.015015,0.006210,0.005260,-0.017769,0.014796,-0.000213
246,0.009002,0.016416,0.000258,0.000352,-0.000917,-0.000074,-0.005269,0.006113
247,0.010714,0.007359,0.014728,0.021743,0.016192,-0.003961,0.004756,0.010725
248,-0.001317,0.007692,0.008500,0.002922,0.005425,-0.004667,0.007241,0.014974


In [21]:
cov_matrix = future_return.cov()
# Number of assets
num_assets = len(mean_returns)

# Objective function: Portfolio volatility
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Constraints: Weights sum to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Bounds for weights
bounds = tuple((0, 1) for asset in range(num_assets))

# Initial guess for weights
initial_guess = num_assets * [1. / num_assets,]

# Optimization function
def minimize_volatility(mean_returns, cov_matrix):
    opt_result = minimize(fun=portfolio_volatility,
                          x0=initial_guess,
                          args=(mean_returns, cov_matrix),
                          method='SLSQP',
                          bounds=bounds,
                          constraints=constraints)
    return opt_result.x

# Calculate optimal weights
optimal_weights = minimize_volatility(mean_returns, cov_matrix)

for stock, weight in zip(stocks, optimal_weights):
    print(f"{stock}: {weight:.20%}")
print(f"The estimate return of the portfolio is: {sum(optimal_weights*mean_returns)}")

WAB: 0.00000000000000000000%
LNT: 8.06025792640135207989%
VZ: 17.93148971318071716041%
TRV: 27.58650410033712319091%
EIX: 0.00000000000000145583%
DHI: 2.82461196224525235010%
WMT: 18.53654177775289824126%
RSG: 25.06059452008266319467%
The estimate return of the portfolio is: 1.16774690219049
